# 통시적 베이즈

1. 베이즈 정리는 이제 다음과 같이 나타낼 수 있다. 
    - $P(A|B) = \frac{P(A)*P(B|A)}{P(B)}$
2. 이 식을 통시적 관점에서 봤을 때, 다음과 같이 H와 D를 써서 나타낼 수 있다.
    - $P(H|D) = \frac{P(H)*P(D|H)}{P(D)}$
    - P(H) = 데이터를 구하기 전 가설의 확률, __사전확률__
    - P(D) = 어떤 가설에서든 해당 데이터가 나올 확률, 데이터의 전체 확률
    - P(D|H) = 가설 하에서 해당 데이터가 나올 확률 ,__가능도(likelihood;우도)__
    - P(H|D) = 데이터를 확인한 후의 가설 확률, __사후확률__

3. 이렇게 이제 사전확률을 통해 사후확률을 업데이트하는 과정을 __베이즈 갱신(Bayesian Update)__ 이라고 한다.    

# 베이즈 테이블

베이즈 갱신을 쉽게 하기 위한 도구로, 베이즈 테이블이 있다. 
다음과 같은 문제에 대해서 베이즈테이블을 작성해보자.

1. 쿠키 그릇 두 개가 있다. 
2. 첫번째 그릇에는 바닐라 쿠키 30개와 초콜릿 쿠키 10개가 들어가 있다.
3. 두번째 그릇에는 바닐라 쿠키와 초콜릿 쿠키가 각각 20개씩 들어가 있다.
4. 어떤 그릇인지 확인하지 않고 임의의 그릇에서 쿠키를 하나 집어서, 그 쿠키가 바닐라쿠키였다면, 이 바닐라 쿠키가 1번 그릇에서 나왔을 확률은 얼마일까?

In [2]:
import pandas as pd

# 한 행에 하나의 가설이 들어갈 빈 테이블
table = pd.DataFrame(index = ['Bowl 1', 'Bowl 2'])
# 사전확률(P(H))을 나타내는 열을 추가
table['prior'] = 1/2,1/2
# 가능도(P(D|H))에 대한 열을 추가
table['likelihood'] = 3/4, 1/2

table

,prior,likelihood
Bowl 1,0.5,0.75
Bowl 2,0.5,0.50


In [3]:
# P(A and B) = P(B)P(A|B)
# unnorm = 표준화되지 않은 사후확률
table['unnorm'] = table['prior'] * table['likelihood']
table

,prior,likelihood,unnorm
Bowl 1,0.5,0.75,0.375
Bowl 2,0.5,0.50,0.250


- 위의 각 값은 베이즈정리의 분자다. 사전확률과 가능도의 곱이다. 
$$P(B)*(P(D|B))$$
- 여기서의 결과를 더하면 다음과 같이 베이즈 정리의 분모가 된다. (전체확률의 정리)
$$P(B_{1})*(P(D|B_{1})) + P(B_{2})*(P(D|B_{2})) =  P(D)$$

- 표준화되지 않은 사후확률들을 더한 값(P(D))으로 각 사후확률 값들을 나눠서 구한 사후확률(P(H|D))을 더하면 1이 되는 것을 알 수 있다.
- 이 과정을 표준화(normalization)라고 하고, 이 때 데이터의 총 확률을 표준화 상수(normalizing constant)라고 한다.

In [4]:
# 전체확률(P(D))
prob_data = table['unnorm'].sum()
prob_data

0.625

In [5]:
# 사후확률(P(H|D))
table['posterior'] = table['unnorm']/ prob_data
table

,prior,likelihood,unnorm,posterior
Bowl 1,0.5,0.75,0.375,0.6
Bowl 2,0.5,0.50,0.250,0.4


# 주사위 문제

- 육면체, 팔면체, 십이면체 주사위가 든 상자가 있다고 가정하자. 이 중 주사위 하나를 임의로 집어서 굴렸더니 1이 나왔다. 이 경우 육면체 주사위를 골랐을 확률은 얼마일까?

In [6]:
# 베이즈테이블을 만들어보자
dice = pd.DataFrame(index = ['Six Dice', 'Eight Dice', 'Twelve Dice'])

# 육면체, 팔면체, 십이면체 주사위를 고를 확률(prior)
dice['prior'] = 1/3,1/3,1/3
# 각 주사위에서 1이 나올 확률(likelihood)
dice['likelihood'] = 1/6, 1/8, 1/12
# unnorm, 표준화되지 않은 사후확률(베이즈정리의 분자), P(B)*(P(D|B))
dice['unnorm'] = dice['prior'] * dice['likelihood']
prob_data = dice['unnorm'].sum()
dice['posterior'] = dice['unnorm'] / prob_data
dice


,prior,likelihood,unnorm,posterior
Six Dice,0.333333,0.166667,0.055556,0.444444
Eight Dice,0.333333,0.125000,0.041667,0.333333
Twelve Dice,0.333333,0.083333,0.027778,0.222222


In [7]:
# 이 과정이 반복적이고 귀찮으므로, 함수로 만들자.

def update(table):
    table['unnorm'] = table['prior'] * table['likelihood']
    prob_data = table['unnorm'].sum()
    table['posterior'] = table['unnorm'] / prob_data
    return table # return 은 솔직히 아무래도 상관없다.

# 몬티홀 문제

- 진행자 몬티홀은 당신에게 1,2,3 이라고 번호가 붙은 세개의 문을 보여주고 각 문 뒤에 상품이 있다고 한다.
- 한 가지 상품은 비싼 것, 나머지 두 상품은 덜 비싼 것이다.
- 게임의 목적은 자동차의 위치를 맞추는 데 있다. 만약 제대로 맞추면, 자동차를 가질 수 있다.

만약 1번 문을 선택했다고 하자, 당신이 고른 문을 열기 전에 몬티는 3번 문을 열어 염소가 있다는 것을 보여줄 것이다.
그 후 몬티는 원래의 선택을 고수할 지, 열리지 않은 나머지 문으로 바꿀 것인지를 다시 묻는다.
자동차를 얻을 기회를 극대화하고자 할 때, 1번 문을 그대로 고수할 것인가, 2번 문으로 바꿀 것인가?

이에 답하려면, 진행자의 행동에 대해 몇 가지를 가정해야 한다.

1. 몬티는 언제나 문을 열고 선택을 바꿀 기회를 준다.
2. 몬티는 당신이 고른 문이나 자동차가 있는 문을 열지 않는다.
3. 당신이 고른 문 뒤에 차가 있는 경우에는 나머지 문을 임의로 연다.

In [8]:
# 각 문에 대한 사전확률은 1/3
from fractions import Fraction

door = pd.DataFrame(index=['Door 1', 'Door 2', 'Door 3'])

door['prior'] = Fraction(1,3)
door

,prior
Door 1,1/3
Door 2,1/3
Door 3,1/3


1. 참가자가 고른 문을 A, 나머지 두 문을 B,C 라 하고 "사회자가 A문 제외의 문을 열어주고 염소가 있음을 보여주는 사건"을 D라고 하자.
2. 우리가 구하고자 하는 것은 사건 D가 일어난 상태에서 A문에 자동차가 있는 사건(이하 사건 A)의 조건부 확률, 즉 $P(A|D)$ 이다. 
3. 이제 베이즈정리에 따라 해당 사건의 역사건, 즉 어떤 문 X안에 자동차가 있을 때 사회자가 C문을 여는 사건(D|X)을 생각해보자.

- 사회자는 A문을 열 수 없고, B,C 중 하나만을 고를 수 있으며 염소가 있는 문을 열어줘야 한다는 목적이 있는 상태이므로 각각의 확률은 다음과 같이 설정이 된다.
    1. 만약 자동차가 A에 있다면, 사회자는 B,C 중 하나를 열어줄 것이다. 2가지 선택지 중 하나를 고르는 사건이므로 P(D|A) = 1/2.
    2. 만약 자동차가 B에 있다면, 사회자는 A,B를 둘 다 열 수 없다. (하나는 참가자가 골랐고, 하나는 자동차가 들어있다.) 따라서 P(D|B) = 1
    3. 만약 자동차가 C에 있다면, 사회자가 C 문을 여는 일은 일어날 수 없다. P(D|C) = 0

In [9]:
# 몬티가 3번 문을 열어 차가 없음을 보여주었으므로, 각 가설 하에서 이 데이터가 나올 확률 likelihood (P(D|X))는 1/2, 1, 0 이 된다.

door['likelihood'] = Fraction(1,2), 1, 0
door

,prior,likelihood
Door 1,1/3,1/2
Door 2,1/3,1
Door 3,1/3,0


In [10]:
update(door)

,prior,likelihood,unnorm,posterior
Door 1,1/3,1/2,1/6,1/3
Door 2,1/3,1,1/3,2/3
Door 3,1/3,0,0,0


이를 통해, 확률에 대한 우리의 직관은 항상 믿을만 하지는 않다는 것을 알 수 있다. 이럴 때 베이즈이론을 통해 다음과 같은 divide-conquer 전략을 사용하면 도움이 된다.

1. 가설과 데이터(사건)를 정리한다.
2. 사전확률을 구한다.
3. 각 가설하에서의 데이터의 가능도를 구한다.

나머지는 베이즈테이블에게 맡기자.

# 연습문제 

## 문제 2-1

- 상자에 두개의 동전이 있다. 하나는 앞뒷면으로 이루어진 일반적인 동전이고, 한 동전은 양쪽이 모두 앞면인 이상한 동전이다. 만약 이 두 동전 중 하나를 임의로 꺼내서 한 면을 살펴보았더니 앞면이었다고 한다면, 이 때 이상한 동전을 골랐을 확률은 얼마인가?

In [11]:
from fractions import Fraction
import pandas as pd

box = pd.DataFrame(index = ['coin 1', 'coin 2'])
# 가설 1, 2에 따른 사전확률
box['prior'] = Fraction(1,2),Fraction(1,2)

print(box)
# 우도(P(D|H)), 각 가설 하에서 데이터가 나올 확률
box['likelihood'] = Fraction(1,2),1
print(box)

# unnorm,posterior
box['unnorm'] = box['prior'] * box['likelihood']
prob_data = box['unnorm'].sum()
box['posterior'] = box['unnorm'] / prob_data

print(box)

       prior
coin 1   1/2
coin 2   1/2
       prior likelihood
coin 1   1/2        1/2
coin 2   1/2          1
       prior likelihood unnorm posterior
coin 1   1/2        1/2    1/4       1/3
coin 2   1/2          1    1/2       2/3


## 문제 2-2

- 누군가를 만나서 그 사람의 자녀가 둘이라는 사실을 알았다. 한 자녀가 딸인지 물었더니 맞다고 했다. 그렇다면 두 자녀 모두 딸일 확률은 얼마인가?

In [12]:
children = pd.DataFrame(index=['child 1', 'child 2'])
children['prior'] = Fraction(1,2),Fraction(1,2)
children['likelihood'] = 1, Fraction(1,2)
# unnorm = P(H)*P(D|H)
children['unnorm'] = children['prior'] * children['likelihood']
prob_data = children['unnorm'].sum()
children['posterior'] = children['unnorm'] / prob_data 
print(children)

        prior likelihood unnorm posterior
child 1   1/2          1    1/2       2/3
child 2   1/2        1/2    1/4       1/3


## 문제 2-3

- 몬티홀문제에는 많은 변형이 있다. 예를 들어, 몬티가 가능한 한 2번 문을 열고, 어쩔 수 없을 때(차가 2번 문 뒤에 있을 때)만 3번문을 연다고 하자.
- 만약 1번 문을 골랐는데 몬티가 2번 문을 열었다면, 차가 3번 문 뒤에 있을 확률은 얼마인가?
- 만약 1번 문을 골랐고 몬티가 3번 문을 열었다면, 차가 2번 문 뒤에 있을 확률은 얼마인가?

In [13]:
monti = pd.DataFrame(index = ['Door 1', 'Door 2', 'Door 3'])
monti['prior'] = Fraction(1,3)
# 1번 문을 골랐는데, 몬티가 2번 문을 열었다면?
# If the car is behind Door 1, Monty would always open Door 2 
# If the car was behind Door 2, Monty would have opened Door 3
# If the car is behind Door 3, Monty would always open Door 2
monti['likelihood'] = 1,0,1
update(monti)
print(monti)

       prior  likelihood unnorm posterior
Door 1   1/3           1    1/3       1/2
Door 2   1/3           0      0         0
Door 3   1/3           1    1/3       1/2


In [14]:
# If the car is behind Door 1, Monty would have opened Door 2
# If the car is behind Door 2, Monty would always open Door 3
# If the car is behind Door 3, Monty would have opened Door 2
monti2 = pd.DataFrame(index = ['Door 1', 'Door 2', 'Door 3'])
monti2['prior'] = Fraction(1,3)
monti2['likelihood'] = 0,1,0
update(monti2)
print(monti2)

       prior  likelihood unnorm posterior
Door 1   1/3           0      0         0
Door 2   1/3           1    1/3         1
Door 3   1/3           0      0         0


## 문제 2-4

- M&M은 여러 색의 설탕이 입혀진 작은 초콜릿이다. M&M을 만드는 마즈사에서는 때때로 색의 조합을 바꿔왔다. 1995년에는 파란색의 M&M을 출시했다.

- 1994년에는 갈색 30%, 노랑 20%, 빨강 20%, 녹색 10%, 주황 10%, 황갈색 10%였다.
- 1996년에는 파란색 24%, 녹색 20%, 주황 16%, 노랑 14%, 빨강 13%, 갈색 13%였다.
- 한 친구가 M&M을 두 봉지 산 다음, 이 중 한 봉지는 1994년에 생산되었고 다른 하나는 1996년에 생산되었다고 알려주었다. 그는 어느 게 무엇인지는 말해주지 않고, 각 봉지에서 M&M을 하나씩 꺼냈다.
- 이 때 하나는 노랑이고 하나는 녹색이었다. 이 때 노랑초콜릿이 1994년에 생산된 봉지에서 나왔을 확률은 얼마일까?

In [16]:
MM = pd.DataFrame(index = ['1994', '1996'])
# Hypothesis:
# A: yellow from 94, green from 96
# A: green from 94, yellow from 96

MM['prior'] = Fraction(1,2)
MM['likelihood'] = 0.2*0.2, 0.14*0.1
update(MM)
print(MM)

     prior  likelihood unnorm posterior
1994   1/2       0.040   0.02  0.740741
1996   1/2       0.014  0.007  0.259259
